# Neptune API tour


## Introduction

This guide will show you how to:

* Install neptune-client
* Connect Neptune to your script and create the first experiment
* Log simple metrics to Neptune and explore the in the UI
* Log learning curves, images and model binaries from Keras training and see those in the Neptune UI
* Fetch the data you logged to Neptune directly into your notebook and analyze them 

By the end of it, you will run your first experiment and see it in Neptune!

## Setup

Install Neptune client

In [ ]:
! pip install --quiet git+https://github.com/neptune-ai/neptune-client.git@alpha

## Initialize Neptune

Connect your script to Neptune application and create new experiment.

In [ ]:
import neptune.alpha as neptune

exp = neptune.init(project='common/colab-test-run',
                   api_token='ANONYMOUS')

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new experiment in Neptune. For now on you will use `exp` to log metadata to it.

---

**Note**


Instead of logging data to the public project 'common/colab-test-run' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs-beta.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of the ``neptune.init()``.

For example:

```python
neptune.init(project_qualified_name='my_workspace/my_project', 
             api_token='MY_API_TOKEN')
```

## Basic Example

Lets start with something super simple.

You will:
* log parameters
* log a training and validation scores
* log loss
* add name to the experiment
* append two tags

In [ ]:
params = {'learning_rate': 0.1}

# log params
exp['parameters'] = params

# log name and append tags
exp["sys/name"] = 'basic-colab-example'
exp["sys/tags"].add(['colab', 'intro'])

# log loss during training
for epoch in range(100):
    exp["train/loss"].log(0.99 ** epoch)

# log train and validation scores
exp['train/accuracy'] = 0.95
exp['valid/accuracy'] = 0.93

Experiments can be viewed as dictionary-like structures - **namespaces** - that you can define in your code. You can appy hierarchical structure to your metadata that will be reflected in the UI as well. Thanks to this you can easily organize your metadata in a way you feel is most convenient.

There is one special namespace: **system namespace**, denoted `sys`. You can use it to add name and tags to the experiment.

Try to change the values and rerun to see your experiments appear in the dashboard.

All `basic-colab-example` experiments are grouped in [this dashboard view](https://alpha.neptune.ai/o/common/org/colab-test-run/experiments?viewId=2eb46477-0064-4617-a182-17f8a2fcc06a).

![colab-experiments](https://neptune.ai/wp-content/uploads/docs-api-tour.png)

There are many other things that you can log to neptune:

* Images and charts
* Artifacts like model weights or results
* Text values
* Hardware consumption
* Code snapshots
* and more

You can go and see all that in the [documentation](https://docs-beta.neptune.ai/user-guides/logging-and-managing-experiment-results/logging-experiment-data) but you can check out the next example to see some of those.

## Keras classification example [Advanced]

Install and import your machine learning libraries

In [ ]:
! pip install --quiet tensorflow==2.3.1 scikit-plot==0.3.7

Get the data

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Prepare the model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

To log metrics after every batch and epoch let's create `NeptuneLogger` callback:

In [ ]:
from tensorflow.keras.callbacks import Callback

class NeptuneLogger(Callback):
    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            exp['batch/{}'.format(log_name)].log(log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            exp['epoch/{}'.format(log_name)].log(log_value)

Now we simply need to create an experiment. 
I will tag it with the name `advanced` and log hyperparameters `epoch_nr` and `batch_size`: 

In [ ]:
EPOCH_NR = 5
BATCH_SIZE = 32

exp = neptune.init(project='common/colab-test-run',
                   api_token='ANONYMOUS')

# log params
exp['parameters/epoch_nr'] = EPOCH_NR
exp['parameters/batch_size'] = BATCH_SIZE

# log name and append tag
exp["sys/name"] = 'keras-metrics'
exp["sys/tags"].add('advanced')

Now we pass our `NeptuneLogger` as keras callback and thats it.

In [ ]:
history = model.fit(x=x_train,
                    y=y_train,
                    epochs=EPOCH_NR,
                    batch_size=BATCH_SIZE,
                    validation_data=(x_test, y_test),
                    callbacks=[NeptuneLogger()])

You can click on the experiment link above and monitor your learning curves as it is training!

![advanced](https://neptune.ai/wp-content/uploads/docs-api-tour-2.png)

Great thing is, you can log more things if you need to during or after the training is finished.

For example, let's calculate some additional metrics on test data and log them.

In [ ]:
import numpy as np

y_test_pred = np.asarray(model.predict(x_test))
y_test_pred_class = np.argmax(y_test_pred, axis=1)

In [ ]:
from sklearn.metrics import f1_score

exp['test/f1'] = f1_score(y_test, y_test_pred_class, average='micro')

We can log diagnostic charts like confusion matrix or ROC AUC curve.

In [ ]:
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix, plot_roc

fig, ax = plt.subplots(figsize=(16, 12))
plot_confusion_matrix(y_test, y_test_pred_class, ax=ax)
exp['diagnostic_charts'].log(neptune.types.Image(fig))

fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y_test, y_test_pred, ax=ax)
exp['diagnostic_charts'].log(neptune.types.Image(fig))

![alt text](https://neptune.ai/wp-content/uploads/docs-api-tour-charts.png)

We can also log model weights to Neptune.

In [ ]:
model.save('my_model.h5')
exp["model"].save('my_model.h5')

![alt text](https://neptune.ai/wp-content/uploads/docs-api-tour-model.png)

With that you can share models with your teammates easily.

You can play around and run this experiment with different parameters and see results and compare them.

Like I've done here:

![alt text](https://neptune.ai/wp-content/uploads/docs-api-tour-compare.png)

A cool thing is, once things are logged to Neptune you can access them from wherever you want.
Let me show you. 



## Access data you logged programatically 

## Getting the project's leaderboard

You can use the Project object to get aggregate data of all your experiments. Here, fetch only experiment with tag 'advanced'.

In [ ]:
my_project = neptune.get_project('common/colab-test-run')
exp_df = my_project.get_experiments_table(tag=['advanced']).as_pandas()
exp_df.head()

## Getting the experiment's metadata

Neptune lets you fetch whatever you logged to it directly to your notebooks and scripts.

In [ ]:
exp = neptune.init(project='common/colab-test-run' ,experiment='COL-7')

Now that your experiment is *fetched* you can download the experiment metadata.

In [ ]:
batch_size = exp["parameters/batch_size"].get()
last_batch_acc = exp['batch/accuracy'].get_last()
print('batch_size: {}'.format(batch_size))
print('last_batch_acc: {}'.format(last_batch_acc))

You can even download artifacts from that experiment if you want to:



In [ ]:
exp['model'].download()

In [ ]:
! ls ./

## Learn more about Neptune

Read about other Neptune features, create your free account and start logging!

[Go to Neptune](https://neptune.ai/register)